In [1]:
import os
import json
import pandas as pd
from casstools.cass_client import CassClient

In [2]:
client = CassClient()

In [3]:
def initialize_directories(basepath="."):
    directories = ['blackboard', 'fetched', 'templates']
    for d in directories:
        pathspec = os.path.join(basepath,d)
        if not os.path.exists(pathspec):
            os.makedirs(pathspec)

In [4]:
def fetch_assignments(course_key, assignment_name) -> pd.DataFrame:
    me = client.whoami()
    assignments = client.get_course_assignments(course_key)
    roster = client.get_course_roster(course_key)
    assignment_dict = assignments[ assignments['assignment_name'] == assignment_name].to_dict(orient="records")[0]
    student_list = roster['student'].tolist()
    folder = f"./fetched/{course_key}/{me['name']}/{assignment_dict['unit_name']}/{assignment_dict['assignment_name']}"
    os.makedirs(folder, exist_ok=True)

    for student_name in student_list:
        info = client.submission_info(course_key, assignment_name, student_name, assignment_name)
        infofile = f"{folder}/{student_name}-info.json"
        with open(infofile,"w") as f:
            f.write(json.dumps(info))
        if info['exists']:
            #fetch the file!
            notebook = client.get_assignment_submission(course_key, assignment_name, student_name, assignment_name)
            #TODO save the notebook file....
            print(notebook)
    return assignment_dict, student_list, me
    

In [5]:
fetch_assignments("tst101-spring2024", "A2.ipynb")

{'cells': [{'cell_type': 'code', 'execution_count': None, 'id': '710a0342-378d-4d8c-8b72-40173b6180aa', 'metadata': {}, 'outputs': [], 'source': []}, {'cell_type': 'code', 'execution_count': None, 'id': '5dd9adf3-3b30-407b-ab65-74ef06728d5f', 'metadata': {}, 'outputs': [], 'source': []}, {'cell_type': 'code', 'execution_count': 1, 'id': '8834732d-38a3-462c-9ee0-d27c4dafb46b', 'metadata': {'tags': []}, 'outputs': [], 'source': ['from casstools.assignment import Assignment\n', 'a = Assignment()']}, {'cell_type': 'code', 'execution_count': 2, 'id': '929c7d4b-945c-4170-ab7c-0faf23f21b1d', 'metadata': {}, 'outputs': [{'data': {'text/plain': ["'spring2024'"]}, 'execution_count': 2, 'metadata': {}, 'output_type': 'execute_result'}], 'source': ['a.parsed_path.term_part']}, {'cell_type': 'code', 'execution_count': 3, 'id': '0dcbced3-1f79-4b3e-8e6c-7f6598aae93f', 'metadata': {'tags': []}, 'outputs': [{'data': {'text/plain': ["['A1.ipynb', 'A2.ipynb']"]}, 'execution_count': 3, 'metadata': {}, 'ou

({'unit_name': 'A',
  'assignment_name': 'A2.ipynb',
  'points': 5,
  'due_date': '2024-01-06 23:59',
  'lms_column_id': 342123},
 ['mafudge@syr.edu', 'dlnosky@syr.edu'],
 {'valid': True,
  'last_activity': '2024-01-07T17:13:53.525136Z',
  'name': 'mafudge@syr.edu',
  'admin': True,
  'kind': 'user',
  'groups': [],
  'fake_token': False})

In [24]:
os.mkdir

<function posix.mkdir(path, mode=511, *, dir_fd=None)>